In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import shared

ve_or_ju = "ju"

In [2]:
order18 = pd.read_csv(f'data/running_order_j2018_{ve_or_ju}.tsv', delimiter="\t")

In [3]:
order18["leg_nro"] = order18["leg"]
order18["orig_name"] = order18["name"]
order18["name"] = order18["name"].str.lower()

In [4]:
order18.head()

,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name
0,1,IFK Göteborg 1,IFK Göteborg,SWE,1,11.0,max peter bejmer,1,Max Peter Bejmer
1,1,IFK Göteborg 1,IFK Göteborg,SWE,2,11.9,johan högstarnd,2,Johan Högstarnd
2,1,IFK Göteborg 1,IFK Göteborg,SWE,3,12.7,vetle ruud bråten,3,Vetle Ruud Bråten
3,1,IFK Göteborg 1,IFK Göteborg,SWE,4,8.8,jonas pilblad,4,Jonas Pilblad
4,1,IFK Göteborg 1,IFK Göteborg,SWE,5,8.7,jens wängdahl,5,Jens Wängdahl


In [5]:
predictions_and_history = pd.read_csv(f"data/preds_for_runners_with_history_2_{ve_or_ju}.csv", delimiter="\t")
predictions_and_history.head()


,Unnamed: 0,mean_team_id,num_valid_times,mean_pace,stdev,prior_mean,prior_log_std,predicted_pace,pred_pace_error,predicted_log_pace_mean,predicted_log_pace_std,name,teams
0,0,18.5,5,5.905,0.177,6.373,0.069,6.333,0.428,1.846,0.066,joni hirvikallio,KOOVEE
1,1,12.0,7,6.051,0.483,6.096,0.043,6.092,0.041,1.807,0.041,topi anjala,KOOVEE
2,2,3.0,7,5.933,0.382,6.244,0.068,6.195,0.262,1.824,0.063,lauri sild,DELTA;KOOVEE;HIIDENKIERTÄJÄT
3,3,18.0,3,6.069,0.363,6.431,0.119,6.396,0.327,1.856,0.114,oleksandr kratov,OK ORION;KOOVEE
4,4,21.0,5,5.936,0.345,6.080,0.071,6.075,0.139,1.804,0.067,kenny kivikas,KANGASALA SK;KOOVEE


In [6]:

predictions_and_history["num_runs"] = predictions_and_history["num_valid_times"]
no_history_row = pd.DataFrame([[0, 0, 0]], columns=["predicted_log_pace_mean", "predicted_log_pace_std", "num_valid_times"])
def get_history_and_preds(running_order_row):
    history_row = shared.get_matching_history_row_for_runner(running_order_row, predictions_and_history, no_history_row)
    #print(f"estimate_row log_means {history_row.log_means} {history_row.log_stdevs}")
    pred_log_mean = history_row.predicted_log_pace_mean.values[0]
    pred_log_std = history_row.predicted_log_pace_std.values[0]
    num_valid_times = history_row.num_valid_times.values[0]
    return pd.Series({"pred_log_mean": pred_log_mean, "pred_log_std": pred_log_std, "num_valid_times": num_valid_times})

history_and_preds = order18.apply(lambda row: get_history_and_preds(row), axis=1)
order18 = order18.assign(num_runs = history_and_preds.num_valid_times)
order18 = order18.assign(pred_log_mean = history_and_preds.pred_log_mean)
order18 = order18.assign(pred_log_std = history_and_preds.pred_log_std)



name 'anssi koirikivi' team_name 'RASTIIMI'
by_name 0 by_name_and_colon 3 runners 2
Duplicate runner       Unnamed: 0  mean_team_id  num_valid_times  mean_pace  stdev  prior_mean  \
1690        1699         210.0                6      7.330  0.618       8.050   
1691        1700         231.0                1      5.943  0.000       8.334   

      prior_log_std  predicted_pace  pred_pace_error  predicted_log_pace_mean  \
1690          0.122           7.876            0.546                    2.064   
1691          0.151           8.279            2.336                    2.114   

      predicted_log_pace_std                               name  \
1690                   0.109           anssi koirikivi:RASTIIMI   
1691                   0.150  anssi koirikivi:RASTIIMI - NONAME   

                  teams  num_runs  
1690           RASTIIMI         6  
1691  RASTIIMI - NONAME         1  
name 'petri laaksonen' team_name 'UNO SK'
by_name 0 by_name_and_colon 6 runners 2
Duplicate runner   

name 'antti jokinen' team_name 'HERMANNIN KÖÖRI'
by_name 0 by_name_and_colon 8 runners 2
Duplicate runner       Unnamed: 0  mean_team_id  num_valid_times  mean_pace  stdev  prior_mean  \
6429        6471        1533.0                1     18.891    0.0      10.861   
6432        6474        1493.0                1     16.689    0.0      12.059   

      prior_log_std  predicted_pace  pred_pace_error  predicted_log_pace_mean  \
6429          0.196          11.094            7.797                    2.406   
6432          0.177          12.154            4.535                    2.498   

      predicted_log_pace_std                               name  \
6429                   0.192      antti jokinen:HERMANNIN KÖÖRI   
6432                   0.174  antti jokinen:DNA HERMANNIN KÖÖRI   

                    teams  num_runs  
6429      HERMANNIN KÖÖRI         1  
6432  DNA HERMANNIN KÖÖRI         1  


In [7]:

(top_countries, top_first_names) = shared.read_persisted_dummy_column_values(ve_or_ju)

In [8]:
order18["year"] = "NA"
features = shared.preprocess_features(order18, top_countries, ve_or_ju)
features.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12972 entries, 0 to 12971
Data columns (total 13 columns):
team_id           12972 non-null int64
team              12972 non-null object
team_base_name    12972 non-null object
team_country      12972 non-null object
leg               12972 non-null int64
leg_dist          12972 non-null float64
name              12972 non-null object
leg_nro           12972 non-null int64
orig_name         12972 non-null object
num_runs          12972 non-null float64
pred_log_mean     12972 non-null float64
pred_log_std      12972 non-null float64
year              12972 non-null object
dtypes: float64(4), int64(3), object(6)
memory usage: 1.3+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1587 entries, 0 to 1586
Data columns (total 3 columns):
first_name           1586 non-null object
fn_pace_class        1587 non-null int64
fn_pace_std_class    1587 non-null int64
dtypes: int64(2), object(1)
memory usage: 37.3+ KB


None

(12972, 54)

In [9]:
features.info()

<class 'pandas.core.sparse.frame.SparseDataFrame'>
RangeIndex: 12972 entries, 0 to 12971
Data columns (total 54 columns):
team_id                  12972 non-null Sparse[int64, nan]
team_id_log10            12972 non-null Sparse[float64, nan]
team_id_square           12972 non-null Sparse[int64, nan]
leg_1                    12972 non-null Sparse[uint8, 0]
leg_2                    12972 non-null Sparse[uint8, 0]
leg_3                    12972 non-null Sparse[uint8, 0]
leg_4                    12972 non-null Sparse[uint8, 0]
leg_5                    12972 non-null Sparse[uint8, 0]
leg_6                    12972 non-null Sparse[uint8, 0]
leg_7                    12972 non-null Sparse[uint8, 0]
c_CZE                    12972 non-null Sparse[uint8, 0]
c_DEN                    12972 non-null Sparse[uint8, 0]
c_ESP                    12972 non-null Sparse[uint8, 0]
c_EST                    12972 non-null Sparse[uint8, 0]
c_FIN                    12972 non-null Sparse[uint8, 0]
c_FRA          

In [10]:

gbr_sd_estimate = shared.predict_without_history(features)

,log_q_low,predicted,log_q_high,log_std
0,1.727,5.944,1.814,0.044
1,1.725,5.982,1.684,-0.020
2,1.722,5.638,1.817,0.048
3,1.694,6.071,1.872,0.089
4,1.669,5.162,1.877,0.104
5,1.666,5.309,1.810,0.072
6,1.683,5.385,1.811,0.064
7,1.737,7.287,2.023,0.143
8,1.741,6.159,1.878,0.069
9,1.743,6.061,1.881,0.069


0.16943760471788158

In [11]:
order18["predicted"] = gbr_sd_estimate["predicted"]
order18["log_q_low"] = gbr_sd_estimate["log_q_low"]
order18["log_q_high"] = gbr_sd_estimate["log_q_high"]
order18["log_std"] = gbr_sd_estimate["log_std"]


In [13]:
order18.head()

,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,year,predicted,log_q_low,log_q_high,log_std
0,1,IFK Göteborg 1,IFK Göteborg,SWE,1,11.0,max peter bejmer,1,Max Peter Bejmer,4.0,1.845,0.076,NA,5.943582,1.726514,1.813598,0.043542
1,1,IFK Göteborg 1,IFK Göteborg,SWE,2,11.9,johan högstarnd,2,Johan Högstarnd,0.0,0.000,0.000,NA,5.981771,1.724960,1.684404,-0.020278
2,1,IFK Göteborg 1,IFK Göteborg,SWE,3,12.7,vetle ruud bråten,3,Vetle Ruud Bråten,5.0,1.816,0.073,NA,5.638084,1.721827,1.817421,0.047797
3,1,IFK Göteborg 1,IFK Göteborg,SWE,4,8.8,jonas pilblad,4,Jonas Pilblad,6.0,1.820,0.068,NA,6.071166,1.693573,1.872321,0.089374
4,1,IFK Göteborg 1,IFK Göteborg,SWE,5,8.7,jens wängdahl,5,Jens Wängdahl,5.0,1.920,0.088,NA,5.162171,1.669116,1.876981,0.103933


In [14]:
order18.tail()

,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,year,predicted,log_q_low,log_q_high,log_std
12967,1901,Suopursuttaret 1,Suopursuttaret,FIN,3,12.7,maria kettu,3,Maria Kettu,1.0,2.250,0.077,NA,9.499017,2.248849,2.405246,0.078198
12968,1901,Suopursuttaret 1,Suopursuttaret,FIN,4,8.8,oskari rannila,4,Oskari Rannila,1.0,2.400,0.026,NA,11.022797,2.357697,2.408835,0.025569
12969,1901,Suopursuttaret 1,Suopursuttaret,FIN,5,8.7,kauko laukkanen,5,Kauko Laukkanen,6.0,2.669,0.171,NA,15.209203,2.419096,2.529718,0.055311
12970,1901,Suopursuttaret 1,Suopursuttaret,FIN,6,10.8,venla rannila,6,Venla Rannila,1.0,2.442,0.022,NA,11.496908,2.398363,2.441586,0.021612
12971,1901,Suopursuttaret 1,Suopursuttaret,FIN,7,15.1,tanja rannila,7,Tanja Rannila,4.0,2.293,0.148,NA,9.197743,2.225596,2.261936,0.018170
